In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot=True)

In [ ]:
(mnist.train.num_examples,
mnist.validation.num_examples,
mnist.test.num_examples)

In [ ]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

x_image = tf.reshape(x, [-1, 28, 28, 1])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy = tf.reduce_mean( -tf.reduce_sum( y_ * tf.log(y_conv), reduction_indices=[1] ))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [ ]:
init = tf.initialize_all_variables()
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
ses = tf.Session()

In [ ]:
ses.run(init)

In [ ]:
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 400 == 0:
        train_accuracy = ses.run(accuracy, feed_dict = {x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
    ses.run(train_step, feed_dict = {x: batch[0], y_: batch[1], keep_prob: 0.5})

res = ses.run(accuracy, feed_dict = {x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})
print('test accuracy %g' % res)

In [ ]:
model = ses.run([W_conv1, b_conv1, W_conv2, b_conv2, W_fc1, b_fc1, W_fc2, b_fc2])

In [ ]:
ses.close()

In [ ]:
todir = 'model-bnns'

import os
if not os.path.exists(todir):
    os.makedirs(todir)

def name(pref, a):
    shape = 'x'.join([str(x) for x in a.shape])
    return 'model-%s-%s' % (pref, shape)
def save(name, a):
    a.ravel().tofile(os.path.join(todir, name))

save(name('h1w', model[0]), model[0].transpose([3,2,0,1]))
save(name('h1b', model[1]), model[1])
save(name('h2w', model[2]), model[2].transpose([3,2,0,1]))
save(name('h2b', model[3]), model[3])
save(name('h3w', model[4]), model[4].reshape([7,7,64,1024]).transpose([3,2,0,1]))
save(name('h3b', model[5]), model[5])
save(name('h4w', model[6]), model[6].transpose())
save(name('h4b', model[7]), model[7])